<a href="https://colab.research.google.com/github/Konstantin5054232/ausbildungsprojekte/blob/main/08_%C3%B6lquelle_standort/%C3%B6lquelle_standort.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Standort für eine Ölquelle auswählen

Die Ölgesellschaft muss entscheiden, wo sie eine neue Bohrung bohren soll.

Wir haben Ölproben in drei Regionen zur Verfügung gestellt. In jeder Region gibt es 10.000 Vorkommen, in denen die Qualität des Öls und das Volumen seiner Reserven gemessen wurden. Es ist notwendig, ein maschinelles Lernmodell zu erstellen, das Ihnen hilft, die Region zu bestimmen, in der der Bergbau den größten Gewinn erzielen wird. Mögliche Gewinne und Risiken müssen mit Bootstrap-Technik analysiert werden.

Aufgabenbedingungen:
- Nur die lineare Regression ist für das Lernen des Modells geeignet (der Rest ist nicht vorhersehbar genug).
- Bei der Erkundung der Region werden 500 Punkte untersucht, von denen mithilfe des maschinellen Lernens die besten 200 für die Entwicklung ausgewählt werden.
- Das Budget für die Entwicklung von Brunnen in der Region beträgt 10 Milliarden Rubel.
- Bei den aktuellen Preisen bringt ein Barrel Rohmaterial 450 Rubel Umsatz. Der Umsatz pro Produkteinheit beträgt 450 Tausend Rubel, da das Volumen in Tausenden von Fässern angegeben ist.
- Nach der Risikobewertung müssen Sie nur die Regionen verlassen, in denen die Verlustwahrscheinlichkeit kleiner als 2.5% ist. Unter ihnen wählen Sie die Region mit dem höchsten durchschnittlichen Gewinn.

# Datenexploration

In [ ]:
# Wir importieren die notwendigen Bibliotheken
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from scipy import stats as st

In [ ]:
# Wir laden Tabellen mit Daten
geo_data_0 = pd.read_csv('/content/geo_data_0.csv')
geo_data_1 = pd.read_csv('/content/geo_data_1.csv')
geo_data_2 = pd.read_csv('/content/geo_data_2.csv')

In [ ]:
# Wir werden die erhaltenen Daten studieren
datas = [geo_data_0, geo_data_1, geo_data_2]
for file in datas:
    display(file.info())
    display(file.shape)
    display(file.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

(100000, 5)

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


...

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

(100000, 5)

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


**Schlussfolgerungen**

Wir sehen, dass jede Tabelle 100.000 Datensätze ungleich Null enthält und 5 Spalten hat. Das Datenformat in der ID-Spalte ist Objekt und in den anderen Spalten Float64. Die Daten in der id-Spalte stören höchstwahrscheinlich das Training des Modells und haben für uns keine Bedeutung.

# Datenaufbereitung und Modelltraining

In [ ]:
# Wir werden die unnötige Spalte entfernen.
geo_data_0 = geo_data_0.drop(['id'] , axis=1)
geo_data_1 = geo_data_1.drop(['id'] , axis=1)
geo_data_2 = geo_data_2.drop(['id'] , axis=1)

## Region 0

In [ ]:
# Wir werden die Zielzeichen und Bedingungen hervorheben
target = geo_data_0['product']
features = geo_data_0.drop(['product'] , axis=1)

# Wir werden die Daten in Stichproben zerlegen.
features_train, features_valid, target_train_0, target_valid_0 = train_test_split(
    features, target, test_size=0.25, random_state=12345)

# Wir prüfen, ob die Daten korrekt verteilt wurden.
print('{:.0%}'.format(target_train_0.shape[0]/target.shape[0]))
print('{:.0%}'.format(target_valid_0.shape[0]/target.shape[0]))

75%
25%


In [ ]:
# Wir trainieren das Modell, machen eine Vorhersage, berechnen den rmse, den Durchschnitt der vorhergesagten Daten.
model = LinearRegression()
model.fit(features_train, target_train_0)
predicted_valid_0 = model.predict(features_valid)
mean_predict_0 = predicted_valid_0.mean()
rmse = (mean_squared_error(target_valid_0, predicted_valid_0)) ** 0.5
        
print('RMSE des vorhergesagten Rohstoffvolumens:', rmse)
print('Durchschnitt des vorhergesagten Rohstoffvolumens:', mean_predict_0)

RMSE des vorhergesagten Rohstoffvolumens: 37.5794217150813
Durchschnitt des vorhergesagten Rohstoffvolumens: 92.59256778438035


## Region 1

In [ ]:
# Wir werden die Zielzeichen und Bedingungen hervorheben
target = geo_data_1['product']
features = geo_data_1.drop(['product'] , axis=1)

# Wir werden die Daten in Stichproben zerlegen.
features_train, features_valid, target_train_1, target_valid_1 = train_test_split(
    features, target, test_size=0.25, random_state=12345)

# Wir prüfen, ob die Daten korrekt verteilt wurden.
print('{:.0%}'.format(target_train_1.shape[0]/target.shape[0]))
print('{:.0%}'.format(target_valid_1.shape[0]/target.shape[0]))

75%
25%


In [ ]:
# Wir trainieren das Modell, machen eine Vorhersage, berechnen den rmse, den Durchschnitt der vorhergesagten Daten.
model = LinearRegression()
model.fit(features_train, target_train_1)
predicted_valid_1 = model.predict(features_valid)
mean_predict_1 = predicted_valid_1.mean()
rmse = (mean_squared_error(target_valid_1, predicted_valid_1)) ** 0.5
        
print('RMSE des vorhergesagten Rohstoffvolumens:', rmse)
print('Durchschnitt des vorhergesagten Rohstoffvolumens:', mean_predict_1)

RMSE des vorhergesagten Rohstoffvolumens: 0.893099286775617
Durchschnitt des vorhergesagten Rohstoffvolumens: 68.728546895446


## Region 2

In [ ]:
# Wir werden die Zielzeichen und Bedingungen hervorheben
target = geo_data_2['product']
features = geo_data_2.drop(['product'] , axis=1)

# Wir werden die Daten in Stichproben zerlegen.
features_train, features_valid, target_train_2, target_valid_2 = train_test_split(
    features, target, test_size=0.25, random_state=12345)

# Wir prüfen, ob die Daten korrekt verteilt wurden.
print('{:.0%}'.format(target_train_2.shape[0]/target.shape[0]))
print('{:.0%}'.format(target_valid_2.shape[0]/target.shape[0]))

75%
25%


In [ ]:
# Wir trainieren das Modell, machen eine Vorhersage, berechnen den rmse, den Durchschnitt der vorhergesagten Daten.
model = LinearRegression()
model.fit(features_train, target_train_2)
predicted_valid_2 = model.predict(features_valid)
mean_predict_2 = predicted_valid_2.mean()
rmse = (mean_squared_error(target_valid_2, predicted_valid_2)) ** 0.5
        
print('RMSE des vorhergesagten Rohstoffvolumens:', rmse)
print('Durchschnitt des vorhergesagten Rohstoffvolumens:', mean_predict_2)

RMSE des vorhergesagten Rohstoffvolumens: 40.02970873393434
Durchschnitt des vorhergesagten Rohstoffvolumens: 94.96504596800489


**Folgerungen**

Nach der Analyse der erhaltenen Daten sehen wir, dass sich die Situation in den Regionen erheblich unterscheidet. In Region 1 gibt es im Vergleich zu den Regionen 0 und 2 eine signifikante Datenstreuung. Dabei ist der Durchschnitt des vorhergesagten Rohstoffvolumens für Region 1 deutlich niedriger als für die übrigen Regionen.

# Vorbereitung auf die Gewinnberechnung

In [ ]:
# Wir berechnen eine ausreichende Menge an Rohstoffen, um eine neue Bohrung zu entwickeln.
one_point_cost = 10000000000 / 200
value_min = one_point_cost / 450000
print('Ausreichende Rohstoffmenge für die Break-Even-Entwicklung eines neuen Bohrlochs: {:.1f} тыс. барр.'.format(value_min))

Ausreichende Rohstoffmenge für die Break-Even-Entwicklung eines neuen Bohrlochs: 111.1 тыс. барр.


In [ ]:
# Wir vergleichen die resultierenden ausreichenden Rohstoffmengen für die Break-Even-Entwicklung 
# mit dem durchschnittlichen Bestand in jeder Region.
print('{:.1%}'.format(mean_predict_0 / value_min))
print('{:.1%}'.format(mean_predict_1 / value_min))
print('{:.1%}'.format(mean_predict_2 / value_min))

83.3%
61.9%
85.5%


Folgerungen

Wir sehen, dass der durchschnittliche Rohstoffvorrat in allen Regionen unter ausreichender Menge an Rohstoffen liegt, um eine neue Bohrung zu entwickeln. Das schlechteste Ergebnis in der Region liegt bei 1: 61,9 Prozent der Genügsamkeit.

# Gewinn- und Risikoberechnung

In [ ]:
# Wir schreiben eine Funktion, um den Gewinn zu berechnen
def revenue(target, probabilities, count):
    probs_sorted = probabilities.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return 450000 * selected.sum() - 10000000000

In [ ]:
# Wir setzen die Indizes in den Zieltabellen zurück und geben das Datenformat der Vorhersageliste in Series an.
target_valid_0 = target_valid_0.reset_index(drop=True)
predicted_valid_0 = pd.Series(predicted_valid_0)

target_valid_1 = target_valid_1.reset_index(drop=True)
predicted_valid_1 = pd.Series(predicted_valid_1)

target_valid_2 = target_valid_2.reset_index(drop=True)
predicted_valid_2 = pd.Series(predicted_valid_2)

In [ ]:
state = np.random.RandomState(42) 

## Region 0

In [ ]:
# Wir berechnen mit Bootstrap den durchschnittlichen Gewinn, die Wahrscheinlichkeit von Verlusten und das Konfidenzintervall von 95%.
values = []
for i in range(1000):
    target_subsample = target_valid_0.sample(500, replace=True, random_state=state)
    probs_subsample = predicted_valid_0[target_subsample.index]
    values.append(revenue(target_subsample, probs_subsample, 200))

values = pd.Series(values)
mean = values.mean()
confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())

print("Durchschnittlicher Gewinn: {:.1f} млн. руб.".format(mean / 1000000))
print("Wahrscheinlichkeit von Verlusten: {:.1%}".format(len(values.loc[values < 0]) / len(values)))
print("95% Konfidenzintervall:", confidence_interval)

Durchschnittlicher Gewinn: 408.8 млн. руб.
Wahrscheinlichkeit von Verlusten: 6.6%
95% Konfidenzintervall: (392159649.2899968, 425520962.867716)


## Region 1

In [ ]:
values = []
for i in range(1000):
    target_subsample = target_valid_1.sample(500, replace=True, random_state=state)
    probs_subsample = predicted_valid_1[target_subsample.index]
    values.append(revenue(target_subsample, probs_subsample, 200))
 
values = pd.Series(values)
mean = values.mean()
confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())

print("Durchschnittlicher Gewinn: {:.1f} млн. руб.".format(mean / 1000000))
print("Wahrscheinlichkeit von Verlusten: {:.1%}".format(len(values.loc[values < 0]) / len(values)))
print("95% Konfidenzintervall:", confidence_interval)

Durchschnittlicher Gewinn: 513.3 млн. руб.
Wahrscheinlichkeit von Verlusten: 1.6%
95% Konfidenzintervall: (499270889.5529853, 527247359.86852205)


## Region 2

In [ ]:
values = []
for i in range(1000):
    target_subsample = target_valid_2.sample(500, replace=True, random_state=state)
    probs_subsample = predicted_valid_2[target_subsample.index]
    values.append(revenue(target_subsample, probs_subsample, 200))
 
values = pd.Series(values)
mean = values.mean()
confidence_interval = st.t.interval(0.95, len(values)-1, values.mean(), values.sem())

print("Durchschnittlicher Gewinn: {:.1f} млн. руб.".format(mean / 1000000))
print("Wahrscheinlichkeit von Verlusten: {:.1%}".format(len(values.loc[values < 0]) / len(values)))
print("95% Konfidenzintervall:", confidence_interval)

Durchschnittlicher Gewinn: 439.4 млн. руб.
Wahrscheinlichkeit von Verlusten: 5.3%
95% Konfidenzintervall: (422258996.4553227, 456502767.2918152)


# Allgemeine Schlussfolgerung

Für die Entwicklung von Bohrlöchern schlagen wir vor, die Region 1 zu wählen. In dieser Region ist die kleinste der drei Regionen der durchschnittliche Rohstoffvorrat. Das Ziel jedes Unternehmens ist jedoch die Gewinnmaximierung. In Region 1 können wir den maximalen Gewinn erzielen, und das Verlustrisiko ist minimal und entspricht der Aufgabenbedingung.